# Hands-on example

In [1]:
import os
import warnings
import sys

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

## Dataset

In [33]:
data_path = "data/wine-quality.csv"
data = pd.read_csv(data_path)

data.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2184,6.4,0.330,0.24,9.8,0.041,29.0,109.0,0.99560,3.29,0.47,10.1,6
3533,6.6,0.220,0.30,14.7,0.045,50.0,136.0,0.99704,3.14,0.37,10.6,6
52,6.2,0.160,0.33,1.1,0.057,21.0,82.0,0.99100,3.32,0.46,10.9,7
2905,7.6,0.310,0.26,1.7,0.073,40.0,157.0,0.99380,3.10,0.46,9.8,5
2854,6.4,0.210,0.28,5.9,0.047,29.0,101.0,0.99278,3.15,0.40,11.0,6
3298,6.3,0.240,0.35,2.3,0.039,43.0,109.0,0.99056,3.34,0.44,11.8,6
2451,6.6,0.560,0.15,10.0,0.037,38.0,157.0,0.99642,3.28,0.52,9.4,5
207,10.2,0.440,0.88,6.2,0.049,20.0,124.0,0.99680,2.99,0.51,9.9,4
4835,5.6,0.295,0.26,1.1,0.035,40.0,102.0,0.99154,3.47,0.56,10.6,6
2088,8.7,0.150,0.30,1.6,0.046,29.0,130.0,0.99420,3.22,0.38,9.8,6


## Tracking experiments

In [40]:
# mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host localhost --port 5000
remote_server_uri = "http://localhost:5000" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)  # or set the MLFLOW_TRACKING_URI in the env

In [41]:
mlflow.tracking.get_tracking_uri()

'http://localhost:5000'

In [42]:
exp_name = "ElasticNet_wine"
mlflow.set_experiment(exp_name)

<Experiment: artifact_location='mlruns/1', experiment_id='1', lifecycle_stage='active', name='ElasticNet_wine', tags={}>

In [43]:
def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


def load_data(data_path):
    data = pd.read_csv(data_path)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, train_y, test_x, test_y


def train(alpha=0.5, l1_ratio=0.5):
    # train a model with given parameters
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    data_path = "data/wine-quality.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        
        mlflow.sklearn.log_model(lr, "model")

In [45]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160693
  R2: 0.12678721972772677
Save to: mlruns/1/c3db2c3b5d9c4323a96c8718d85b3233/artifacts


In [46]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7859129997062341
  MAE: 0.6155290394093895
  R2: 0.20224631822892103
Save to: mlruns/1/e75fa0286f4446cf894db8d1f6fe243b/artifacts


In [47]:
train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7792546522251949
  MAE: 0.6112547988118587
  R2: 0.2157063843066196
Save to: mlruns/1/a264ed459757429f8f4c03331e9333bb/artifacts


## Getting runs

In [48]:
from datetime import datetime
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiments = client.list_experiments() # returns a list of mlflow.entities.Experiment
print(experiments)

[<Experiment: artifact_location='mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>, <Experiment: artifact_location='mlruns/1', experiment_id='1', lifecycle_stage='active', name='ElasticNet_wine', tags={}>]


In [50]:
# get the run
_run = client.get_run(run_id="a264ed459757429f8f4c03331e9333bb")
print(_run)

<Run: data=<RunData: metrics={'mae': 0.6112547988118587,
 'r2': 0.2157063843066196,
 'rmse': 0.7792546522251949}, params={'alpha': '0.1', 'l1_ratio': '0.1'}, tags={'mlflow.log-model.history': '[{"run_id": "a264ed459757429f8f4c03331e9333bb", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2022-03-14 20:03:38.203895", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"loader_module": "mlflow.sklearn", '
                             '"python_version": "3.8.10", "env": '
                             '"conda.yaml"}, "sklearn": {"pickled_model": '
                             '"model.pkl", "sklearn_version": "1.0.2", '
                             '"serialization_format": "cloudpickle"}}, '
                             '"model_uuid": '
                             '"8345523704e6478cba6c164d7b7cedff"}]',
 'mlflow.source.name': '/home/ritikaiyer/ml/my_env/lib/